In [0]:
!pip install openai==0.28

In [0]:
dbutils.library.restartPython()

In [0]:
#import openai
from pathlib import Path
import os
import pandas as pd
from ParsingMemoriaCalculo import tabela_texto_memoria_para_df
'''openai.api_type = "azure"
openai.api_base = "https://oai-dk.openai.azure.com/"
openai.api_version = "2023-12-01-preview"
openai.api_key = dbutils.secrets.get('akvdesafiokinea','azure-oai-dk')'''

In [0]:
#Função para Gerar Restrição
#Incluir contexto de regra do tipo vedação (ex: 1181) no contexto do GPT
def gerar_restricao(descricao,ativos,saldo_base,exposicao,l_max,l_min):   
    contexto_gpt = '''
        Você irá receber a descrição de uma regra, um conjunto posições de ativos que são considerados para o cálculo dessa regra, um saldo base, limite minimo e máximo em relação a esse saldo base e a porcentagem que não por essa restrição. Lembre-se de considerar se a porcentagem está acima do limite_maximo ou abaixo do limite minimo para tomar a decisão de como escrever o código
        A partir disso você deve gerar um código de restrição no formato utilizado pelo scipy.minimize.
        
        Exemplo 1:
            Descrição: Art. 45. Cumulativamente aos limites de concentração por emissor, a classe de cotas deve observar os seguintes limites de concentração por modalidade de ativo financeiro, sem prejuízo das normas aplicáveis ao seu tipo: I - ate 20% (vinte por cento) de ativos de renda fixa;
            Posição Ativos: [0,1,4,5,6]
            Saldo Base: 500000
            Limite Mínimo: 0
            Limite Máximo: 0.2
            Porcentagem:0.25
            def restricao(x):
                ativos_considerados = [x[0],x[1],x[4],x[5],x[6]...]
                return  0.2 * 500000 -  sum(ativos_considerados)
        Exemplo 2: 
            Descrição: Regulamento - POLÍTICA DE INVESTIMENTO - O objetivo do FUNDO é aplicar, no mínimo, 80% (oitenta por cento) de seus recursos em ativos financeiros de renda fixa relacionados diretamente, ou sintetizados via derivativos, ao fator de risco que dá nome à classe, observado que a rentabilidade do FUNDO será impactada pelos custos e despesas do FUNDO, inclusive taxa de administração.
            Posição Ativos: [0,1,4,5,6]
            Saldo Base: 500000
            Limite Mínimo: 0.8
            Limite Máximo: 1
            Porcentagem: 0.68
            def restricao(x):
                ativos_considerados = [x[0],x[1],x[4],x[5],x[6]...]
                return sum(ativos_considerados) - 0.8 * 500000    
        Exemplo 3:
        Descrição: Vedação
        Posição Ativos: [0,1,4,5,6]
        Saldo Base: 500000
        Limite Mínimo: null
        Limite Máximo: null
        Porcentagem: null
        def restricao(x):
            ativos_considerados = [x[0],x[1],x[4],x[5],x[6]...]
            return -sum(ativos_considerados)
    '''
    message_text = [
                {"role":"system","content":contexto_gpt},
                {"role":"user","content":f"Descrição: {descricao}\n Posições Ativos: {ativos} \n Saldo Base: {saldo_base} \n Limite Mínimo {l_min}  \n Limite Máximo: {l_max} \n Porcentagem: {exposicao}"}
               ]
    
    completion = openai.ChatCompletion.create(
        engine="gpt35turbo16k",
        messages = message_text,
        temperature=0.0,
        max_tokens=1200,
        top_p=0.95,
        frequency_penalty=0.0,
        presence_penalty=0.0,
        stop=None,
        )

    return completion.to_dict()['choices'][0]['message']['content']

In [0]:
%sql
WITH UltimoHistorico AS (
            SELECT '2025-02-10T11:12:37.000+00:00' AS UltimaData
            FROM desafio_kinea.boletagem_cp.nxenq_resultadoenquadramentohist
            WHERE status = 1
        )

        SELECT 
            nxe.IdFundo as IdFundo,
            nxe.IdRegra AS IdRegra_resultado_enquadramento,  
            nxe.DataHoraVersao as DataHoraVersao,
            nxe.SaldoBaseCalculo as SaldoBaseCalculo,
            nxe.ValorExposicao as ValorExposicao,
            nxe.SaldoObjeto as SaldoObjeto,
            nxe.LimiteMin as LimiteMin,
            nxe.LimiteMax as LimiteMax,
            nxr.ValorMin as ValorMin_nexusregras,
            nxr.ValorMax as ValorMax_nexusregras,
            nxr.Descricao as Descricao_nexusregras,
            nxr.DescricaoDetalhada as DescricaoDetalhada_nexusregras,
            nxrc.LimiteMin as LimiteMin_concentracao,
            nxrc.LimiteMax as LimiteMax_concentracao,
            nxrc.Descricao as Descricao_concentracao,
            nxrc.DescricaoDetalhada as DescricaoDetalhada_concentracao,
            nxmc.MemoriaCalculo as MemoriaCalculo
        FROM desafio_kinea.boletagem_cp.nxenq_resultadoenquadramentohist nxe 
        INNER JOIN UltimoHistorico uh 
            ON nxe.DataHoraVersao = uh.UltimaData -- join para filtrar apenas para o último timestamp de simulação (nxe.DataHoraVersao)
        LEFT JOIN desafio_kinea.boletagem_cp.nxenq_regras nxr 
            ON nxr.IdRegra = CAST(regexp_replace(nxe.IdRegra, '[^0-9]', '') AS INT) -- join para a tabela de regras do tipo L%
            AND nxe.IdRegra LIKE 'L%'  
        LEFT JOIN desafio_kinea.boletagem_cp.nxenq_regrasporconcentracao nxrc
            ON nxrc.IdRegra = CAST(regexp_replace(nxe.IdRegra, '[^0-9]', '') AS INT)
            AND (nxe.IdRegra LIKE 'CM%' OR nxe.IdRegra LIKE 'CD%') -- join para a tabela de regras de concentração (CM%/CD%)
        LEFT JOIN desafio_kinea.boletagem_cp.nxenq_memoriacalculo nxmc --join com tabela de memoria de cálculo
            ON nxmc.DataHoraVersao = uh.UltimaData --join pela DataHoraVersao(última simulação), pelo Id do fundo e Id da regra
            AND nxe.IdRegra = nxmc.IdRegra
            AND nxe.IdFundo = nxmc.IdFundo
        where nxe.status = 1 and nxe.IdFundo = 'CP4';

In [0]:
import pandas as pd
#df = pd.read_csv('/Volumes/desafio_kinea/boletagem_cp/files/InputNexxus/input_nexxus_teste_joao.csv',sep='\t')
with open(file='/Volumes/desafio_kinea/boletagem_cp/files/InputNexxus/input_nexxus_teste_joao.csv',mode='r') as file:
    print(file.read())
    

In [0]:
%sql
select * from desafio_kinea.boletagem_cp.nxenq_resultadoenquadramentohist where DataHoraVersao='2025-02-10T11:12:37.000+00:00' and IdRegra = 'L1206'

In [0]:
711.723.122,4708
m = spark.sql("select * from desafio_kinea.boletagem_cp.nxenq_memoriacalculo where  DataHoraVersao='2025-02-10T11:12:37.000+00:00' and IdFundo = 'CP4' and IdRegra = 'L1206'").toPandas()['MemoriaCalculo'][0]
df = tabela_texto_memoria_para_df(m)
df.groupby('NOME',as_index=False).agg({'POSIÇÃO':'sum'}).display()

In [0]:
%sql
select * from desafio_kinea.boletagem_cp.nxenq_memoriacalculo where  DataHoraVersao ='2025-02-12T14:46:18.000+00:00' and IdFundo = 'CP3'

In [0]:
from ParsingMemoriaCalculo import tabela_texto_memoria_para_df
memoria = spark.sql("select * from desafio_kinea.boletagem_cp.nxenq_memoriacalculo where  DataHoraVersao ='2025-02-12T14:46:18.000+00:00' and IdFundo = 'CP3'").toPandas()['MemoriaCalculo'][0]
tabela_memoria = tabela_texto_memoria_para_df(memoria).groupby('NOME',as_index=False).agg({'POSIÇÃO':'sum','NOME':'first'})
ativos = ['DEBN AEGPA0 H750651','DEBN AEGPA3 I109165','DEBN AEGPA7 I445412']
tabela_memoria = tabela_memoria[~tabela_memoria['NOME'].isin(ativos)]
tabela_memoria['POSIÇÃO'].sum()

In [0]:
#Testando Função Regra L636
df_teste = spark.sql("select IdRegra, SaldoBaseCalculo,SaldoObjeto,LimiteMin,LimiteMax,ValorExposicao from desafio_kinea.boletagem_cp.nxenq_resultadoenquadramentohist where IdFundo = 'CP4' and IdRegra = 'L636' and DataHoraVersao = '2025-02-07T11:22:56.000+00:00'").toPandas()

df_teste['DescricaoDetalhada'] = spark.sql("select DescricaoDetalhada from desafio_kinea.boletagem_cp.nxenq_regras where IdRegra = 636").toPandas()['DescricaoDetalhada'][0]

posicao_ativos = [0,2,5,8,9] #Iremos conseguir a partir da memória de cálculo 


resposta = gerar_restricao(df_teste['DescricaoDetalhada'][0],posicao_ativos,df_teste['SaldoBaseCalculo'][0],df_teste['LimiteMin'][0],df_teste['LimiteMax'][0],df_teste['ValorExposicao'][0])

#Testando como adicionar um numero a função
resposta = resposta[:13]+"_1"+resposta[13:]

In [0]:
#Testando Função Regra L153
df_teste = spark.sql("select IdRegra, SaldoBaseCalculo,SaldoObjeto,LimiteMin,LimiteMax,ValorExposicao from desafio_kinea.boletagem_cp.nxenq_resultadoenquadramentohist where IdFundo = 'CP8' and IdRegra = 'CD153' and status='1' and DataHoraVersao = '2025-02-10T11:12:37.000+00:00'").toPandas()

df_teste['DescricaoDetalhada'] = spark.sql("select DescricaoDetalhada from desafio_kinea.boletagem_cp.nxenq_regrasporconcentracao where IdRegra = 153").toPandas()['DescricaoDetalhada'][0]

posicao_ativos = [2,6] #Iremos conseguir a partir da Memória de Cálculo

resposta = gerar_restricao(df_teste['DescricaoDetalhada'][0],posicao_ativos,df_teste['SaldoBaseCalculo'][0],df_teste['LimiteMin'][0],df_teste['LimiteMax'][0],df_teste['ValorExposicao'][0])
print(resposta)

In [0]:
#Testando Função Regra CD156
df_teste = spark.sql("select IdRegra, SaldoBaseCalculo,SaldoObjeto,LimiteMin,LimiteMax,ValorExposicao from desafio_kinea.boletagem_cp.nxenq_resultadoenquadramentohist where IdFundo = 'KCP' and IdRegra = 'CD156' and status='1' and DataHoraVersao = '2025-02-10T11:44:27.000+00:00'").toPandas()

df_teste['DescricaoDetalhada'] = spark.sql("select DescricaoDetalhada from desafio_kinea.boletagem_cp.nxenq_regrasporconcentracao where IdRegra = 156").toPandas()['DescricaoDetalhada'][0]

posicao_ativos = [2,6] #Iremos conseguir a partir da Memória de Cálculo
resposta = gerar_restricao(df_teste['DescricaoDetalhada'][0],posicao_ativos,df_teste['SaldoBaseCalculo'][0],df_teste['LimiteMin'][0],df_teste['LimiteMax'][0],df_teste['ValorExposicao'][0])
print(resposta)


In [0]:
Descricao = "Vedação - LE - CMN 4.661 - Art. 21 § 2º - RF Compromissadas Lastro Privado"
posicao_ativos = [2]
resposta = gerar_restricao(Descricao,posicao_ativos,500000,None,None,0)
print(resposta)

In [0]:
%sql
select distinct IdRegra from desafio_kinea.boletagem_cp.nxenq_resultadoenquadramentohist where status = 1 order by IdRegra

In [0]:
%sql
select * from desafio_kinea.boletagem_cp.nxenq_regrasporconcentracao 

In [0]:
%sql
select * from desafio_kinea.boletagem_cp.nxenq_regras

In [0]:
%sql
select distinct IdRegra from desafio_kinea.boletagem_cp.nxenq_memoriacalculo 